In [377]:
from selenium import webdriver
from time import sleep
import threading
import re
from tqdm import tnrange, tqdm_notebook

In [50]:
def get_item(page, driver): 
    
    # regex of item number
    regex = re.compile('[0-9]+\?page')
    
    driver.get("http://getitbeauty.m.tving.com/Product?page={}".format(page))
    sleep(0.5)

    lis = driver.find_elements_by_css_selector('.subType5 li dl') 

    
    idx = 0
    rows = []
    while(True):
        try: 
            li = lis[idx]
            
            img = li.find_element_by_css_selector('.thumb a img').get_attribute('src')
            name = li.find_element_by_css_selector('dt a').text

            brand = li.find_element_by_css_selector('dt a strong').text

            # split category
            cates = li.find_elements_by_css_selector('.gray a')
            cate_res = []
            for cate in cates:
                cate_res.append(cate.text)


            link = li.find_element_by_css_selector('.thumb a').get_attribute('href')

            # split item number
            matchobj = regex.search(link)
            item_num = matchobj.group()
            item_num = item_num.split('?')[0] 
             
            rows.append([item_num, img, name, brand, cate_res, link])
            
            idx = idx + 1
            if idx == len(lis):
                break
                
        except Exception as e:
            del rows[:]
            rows = [] 
            sleep(1) 
            
            lis = driver.find_elements_by_css_selector('.subType5 li dl') 
            idx = 0 
         
         
    return rows


def get_page(start, end):
    
    driver = webdriver.PhantomJS()
    for page in range(start, end):
        page_items = get_item(page, driver)
        all_items.extend(page_items)
        
    driver.close() 

In [51]:
!killall phantomjs

No matching processes belonging to you were found


431페이지마다 12개 아이템이 있으며 마지막 432페이즈는 8개만 있다.

In [78]:
(431 * 12) + 8

5180

스레드 6개로 분할하여 72페이지씩 크롤링 작업 수행

In [49]:
432 / 6

72.0

In [52]:
all_items = []
for cnt in range(1, 7): 
    start = 72 * (cnt-1)+1
    end = 72 * cnt+1 
    
    th = threading.Thread(target=get_page, args=(start, end))
    th.start()

In [77]:
len(all_items)

5180

In [83]:
item_df = pd.DataFrame(all_items, columns=['id_item','link','name','brand','category','img'])

In [89]:
item_df['cate1']= item_df['category'].apply(lambda x: x[0])
item_df['cate2']= item_df['category'].apply(lambda x: x[1])

In [92]:
item_df = item_df.drop(columns=['category'])

In [94]:
item_df.to_csv('data/item.csv', index=False)

In [307]:
item_df.head()

,id_item,link,name,brand,img,cate1,cate2
0,27111,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 브라이트닝 스팟 컨트롤 파...,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,MAKEUP,파운데이션
1,27110,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 브라이트닝 스팟 컨트롤 베...,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,MAKEUP,메이크업베이스
2,27109,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 브라이트닝 프로텍티브 크림...,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,SKINCARE,미백크림
3,27108,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 인텐시브 스팟 타겟팅 세럼,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,SKINCARE,화이트닝 에센스
4,27107,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 인텐시브 브라이트닝 마스크,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,SKINCARE,마스크팩


In [399]:
def get_item_detail(item, driver):
    driver.get("http://getitbeauty.m.tving.com/Product/ProductView/{}".format(item))
    sleep(0.5)
 
    # 겟잇뷰티 정보 있는지
    getbu_info = driver.find_elements_by_css_selector('.getbu_list ul li a')
    
    rows = []
    while(True):
        try:  
            item_info = driver.find_element_by_css_selector('.prd_info_box') 
            price = driver.find_element_by_css_selector('.prd_info_box .price').text
            getbu_info = driver.find_elements_by_css_selector('.getbu_list ul li a')
             
            # 제품설명
            t = item_info.text.split('\n')
            t = t[len(t)-1].strip()
  
            # 겟잇뷰티 정보 있는지 확인
            link = ''
            g_info = ''
            if len(getbu_info) > 0:
                getbu_info = getbu_info[0]
                link = getbu_info.get_attribute('href')
                g_info = getbu_info.text
                
            rows.append([item, price, t, link, g_info])
            
            break
        except Exception as e:
            #print(e)
            print(item, t, w, p)
            del rows[:]
            rows = [] 
            sleep(1)  
            item_info = driver.find_element_by_css_selector('.prd_info_box') 
            price = driver.find_element_by_css_selector('.prd_info_box .price').text
            getbu_info = driver.find_elements_by_css_selector('.getbu_list ul li a')
    return rows

def get_item_detail_group(items): 
    driver =  webdriver.PhantomJS()
    for item in items: 
        item_detail = get_item_detail(item, driver)
        all_items_detail.extend(item_detail)
    driver.close() 

In [299]:
driver =  webdriver.PhantomJS()

In [309]:
get_item_detail(30551, driver)

[[30551, '9g', '12,000', '컬러의 선명한 발색, 광택 및 지속력을 높여주는 모디 탑 젤', '', '']]

In [391]:
all_items_detail= []
get_item_detail_group(item_df['id_item'].values)

KeyboardInterrupt: 

In [548]:
!killall phantomjs

No matching processes belonging to you were found


In [549]:
all_items_detail= []
for cnt in range(1, 11): 
    start = 518 * (cnt-1)
    end = 518 * cnt-1
    items = item_df.loc[start:end]['id_item'].values
    th = threading.Thread(target=get_item_detail_group, args=(items,))
    th.start()

Exception in thread Thread-95:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-399-84bf341adc7d>", line 42, in get_item_detail_group
    driver =  webdriver.PhantomJS()
  File "/usr/local/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py", line 51, in __init__
    self.service.start()
  File "/usr/local/lib/python3.6/site-packages/selenium/webdriver/common/service.py", line 88, in start
    raise WebDriverException("Can not connect to the Service %s" % self.path)
selenium.common.exceptions.WebDriverException: Message: Can not connect to the Service phantomjs

Exception in thread Thread-91:
Traceback (most recent call last):
  File "/usr

In [578]:
len(all_items_detail)

5180

In [579]:
detail_df = pd.DataFrame(all_items_detail,columns=['id_item','price','info','gb_link','gb_info'])

In [581]:
!ls data

item.csv


In [582]:
detail_df.to_csv('data/item_detail.csv',index=False)